# Imports

In [1]:
import asyncio
import datetime
import json
import os

import lancedb
import pandas as pd
import tiktoken
import torch
import torch.nn.functional as F
from datasets import load_dataset
from dotenv import load_dotenv
from loguru import logger
from openai import OpenAI, AsyncOpenAI
from sentence_transformers import SentenceTransformer
from tenacity import (
    RetryCallState,
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

try:
    from google.colab import userdata  # type: ignore
except ImportError:
    userdata = None

load_dotenv(override=True)
pd.set_option("display.max_colwidth", 0)

# Preparing dataset

In [2]:
# https://huggingface.co/datasets/AiresPucrs/tmdb-5000-movies
ds = load_dataset("AiresPucrs/tmdb-5000-movies", split="train")
ds

Dataset({
    features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
    num_rows: 4803
})

In [3]:
# Remove missing overview
ds = ds.filter(lambda x: bool(x["overview"]))
ds

Dataset({
    features: ['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'cast', 'crew'],
    num_rows: 4800
})

In [4]:
def preprocess(example: dict) -> dict:
    example["genres"] = [g["name"] for g in json.loads(example["genres"])]
    example["keywords"] = [k["name"] for k in json.loads(example["keywords"])]
    example["release_date"] = datetime.datetime.strptime(
        d if (d := example["release_date"]) else "1970-01-01", "%Y-%m-%d"
    ).date()
    example["release_year"] = example["release_date"].year
    example["spoken_languages"] = [
        sl["name"] for sl in json.loads(example["spoken_languages"])
    ]
    example["cast"] = [
        {
            "name": c["name"],
            "character": c["character"],
        }
        for c in json.loads(example["cast"])
    ]
    # example["production_companies"] = [
    #     pc["name"] for pc in json.loads(example["production_companies"])
    # ]
    # example["production_countries"] = [
    #     pc["name"] for pc in json.loads(example["production_countries"])
    # ]
    # example["crew"] = [
    #     {
    #         "name": c["name"],
    #         "job": c["job"],
    #     }
    #     for c in json.loads(example["crew"])
    # ]
    return example


In [5]:
ds = ds.map(
    preprocess,
    remove_columns=[
        "id",
        "homepage",
        "production_companies",
        "production_countries",
        "status",
        "tagline",
        "vote_count",
        "vote_average",
        "crew",
        "original_title",
    ],
    num_proc=4,
)
ds[0]

{'budget': 4000000,
 'genres': ['Crime', 'Comedy'],
 'keywords': ['hotel',
  "new year's eve",
  'witch',
  'bet',
  'hotel room',
  'sperm',
  'los angeles',
  'hoodlum',
  'woman director',
  'episode film'],
 'original_language': 'en',
 'overview': "It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'popularity': 22.87623,
 'release_date': datetime.date(1995, 12, 9),
 'revenue': 4300000,
 'runtime': 98.0,
 'spoken_languages': ['English'],
 'title': 'Four Rooms',
 'cast': [{'character': 'Ted the Bellhop', 'name': 'Tim Roth'},
  {'character': 'Man', 'name': 'Antonio Banderas'},
  {'character': 'Angela', 'name': 'Jennifer Beals'},
  {'character': 'Elspeth', 'name': 'Madonna'},
  {'character': 'Margaret', 'name': 'Marisa Tomei'},
  {'character': 'Leo', 'name': 'Bruce Willis'},
  {'character': 'Cheste

# Embeddings

In [6]:
queries = [
    "I want to watch an exciting superhero movie",
    "我想看一部超級英雄電影",
]

movies = [
    "A movie about a group of friends who go on a road trip",
    "A romantic comedy about a couple who meet at a wedding",
    "An autobiography of George Washington, the first president of the United States",
    "Spider-Man is fighting against the Green Goblin in another universe",
]

## sentence-transformers

* [Official Documentation](https://sbert.net/)
* Models supporting `sentence-transformers`: https://huggingface.co/models?library=sentence-transformers
* Models for `sentence-similarity`: https://huggingface.co/models?pipeline_tag=sentence-similarity


In [11]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)

### Compute embeddings

In [ ]:
query_embeddings = embedder.encode(queries)
movie_embeddings = embedder.encode(movies)
print(query_embeddings.shape, movie_embeddings.shape)
print(f"Query: {query_embeddings[0][:5]}")
print(f"Movie: {movie_embeddings[0][:5]}")

(2, 384) (4, 384)
Query: [-0.4084644  -0.24543129 -0.600559   -0.11258549  0.19218642]
Movie: [ 0.30984214 -0.12824544 -0.33090717 -0.2385715   0.33415732]


### Computing similarity + retrieving top k

In [ ]:
# Compute cosine similarities
similarities = embedder.similarity(query_embeddings, movie_embeddings)
similarities

tensor([[ 0.2611,  0.1753, -0.0321,  0.3556],
        [ 0.2784,  0.1435,  0.0242,  0.3834]])

In [12]:
YELLOW = "\033[33m"
END = "\033[0m"
# Output the pairs with their score
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j, sentence2 in enumerate(movies):
        print(f" - {sentence2: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - A movie about a group of friends who go on a road trip: 0.2611
 - A romantic comedy about a couple who meet at a wedding: 0.1753
 - An autobiography of George Washington, the first president of the United States: -0.0321
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3556
我想看一部超級英雄電影
 - A movie about a group of friends who go on a road trip: 0.2784
 - A romantic comedy about a couple who meet at a wedding: 0.1435
 - An autobiography of George Washington, the first president of the United States: 0.0242
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3834


In [141]:
torch.topk(similarities[0], k=4)

torch.return_types.topk(
values=tensor([ 0.3556,  0.2611,  0.1753, -0.0321]),
indices=tensor([3, 0, 1, 2]))

In [13]:
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3556
 - A movie about a group of friends who go on a road trip: 0.2611
 - A romantic comedy about a couple who meet at a wedding: 0.1753
 - An autobiography of George Washington, the first president of the United States: -0.0321
我想看一部超級英雄電影
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3834
 - A movie about a group of friends who go on a road trip: 0.2784
 - A romantic comedy about a couple who meet at a wedding: 0.1435
 - An autobiography of George Washington, the first president of the United States: 0.0242


## OpenAI

[Documentation](https://platform.openai.com/docs/guides/embeddings?lang=python)

![](https://i.redd.it/lpf0u9nbj7w41.jpg)

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key and userdata:
    # If running in Google Colab, try to get the API key from userdata
    api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = AsyncOpenAI(api_key=api_key, max_retries=5)  # async instead of sync

### Compute embeddings

In [15]:
res = await client.embeddings.create(input=queries[0], model="text-embedding-3-small")
embedding = res.data[0].embedding
print(len(embedding))
print(embedding[:5])

1536
[-0.010025657713413239, 0.007190898526459932, -0.08766952902078629, 0.010150458663702011, -0.025221630930900574]


In [16]:
print(f"Total tokens: {res.usage.total_tokens}")

Total tokens: 8


In [17]:
# Convert to torch.Tensor for cosine similarity
_query_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=query, model="text-embedding-3-small")
        for query in queries
    ]
)
query_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _query_embeddings]
)
_movie_embeddings = await asyncio.gather(
    *[
        client.embeddings.create(input=movie, model="text-embedding-3-small")
        for movie in movies
    ]
)
movie_embeddings = torch.Tensor(
    [embedding.data[0].embedding for embedding in _movie_embeddings]
)

In [18]:
torch.Tensor(movie_embeddings)

tensor([[ 0.0125,  0.0218, -0.0479,  ...,  0.0026, -0.0386,  0.0079],
        [-0.0326,  0.0203, -0.0673,  ...,  0.0057, -0.0081, -0.0111],
        [ 0.0083, -0.0117,  0.0179,  ...,  0.0175, -0.0041, -0.0108],
        [-0.0362, -0.0462,  0.0150,  ..., -0.0224,  0.0127,  0.0131]])

### Computing similarity + retrieving top k

In [ ]:
similarities = F.cosine_similarity(
    query_embeddings.unsqueeze(1), movie_embeddings.unsqueeze(0), dim=2
)
similarities

tensor([[0.2881, 0.2119, 0.0501, 0.3344],
        [0.2241, 0.1764, 0.0754, 0.2568]])

In [21]:
YELLOW = "\033[33m"
END = "\033[0m"
for idx_i, sentence1 in enumerate(queries):
    print(sentence1)
    for idx_j in torch.topk(similarities[idx_i], k=4).indices:
        print(f" - {movies[idx_j]: <30}: {YELLOW}{similarities[idx_i][idx_j]:.4f}{END}")

I want to watch an exciting superhero movie
 - Spider-Man is fighting against the Green Goblin in another universe: 0.3344
 - A movie about a group of friends who go on a road trip: 0.2881
 - A romantic comedy about a couple who meet at a wedding: 0.2119
 - An autobiography of George Washington, the first president of the United States: 0.0501
我想看一部超級英雄電影
 - Spider-Man is fighting against the Green Goblin in another universe: 0.2568
 - A movie about a group of friends who go on a road trip: 0.2241
 - A romantic comedy about a couple who meet at a wedding: 0.1764
 - An autobiography of George Washington, the first president of the United States: 0.0754


### Calculate tokens and price

[OpenAI pricing](https://platform.openai.com/docs/pricing)

In [24]:
enc = tiktoken.encoding_for_model("text-embedding-3-small")

In [27]:
encoded = enc.encode(queries[0])
print(f"Total tokens: {len(encoded)}")
print(encoded)

Total tokens: 8
[40, 1390, 311, 3821, 459, 13548, 46244, 5818]


In [28]:
# price per 1M tokens
model_to_price = {
    "text-embedding-3-small": 0.02,
    "text-embedding-3-large": 0.13,
}


def get_token_count_and_price(
    texts: list[str], model: str = "text-embedding-3-small"
) -> tuple[int, float]:
    if model not in model_to_price:
        raise ValueError(f"Model {model} not supported")
    enc = tiktoken.encoding_for_model(model)
    token_count = sum(len(e) for e in enc.encode_batch(texts))
    price_per_1m_tokens = model_to_price[model]
    price = (token_count / 1_000_000) * price_per_1m_tokens
    return token_count, price

In [29]:
get_token_count_and_price(movies * 10, model="text-embedding-3-small")

(480, 9.600000000000001e-06)

### Handling rate limits

In [ ]:
def log_backoff_attempt(retry_state: RetryCallState) -> None:
    """
    Logs a message before a retry attempt, detailing the attempt number,
    the exception, and the wait time.
    """
    attempt_num = retry_state.attempt_number
    exception = retry_state.outcome.exception() if retry_state.outcome else "N/A"
    wait_time = retry_state.next_action.sleep if retry_state.next_action else 0.0
    func_name = retry_state.fn.__name__ if retry_state.fn else "N/A"

    logger.info(
        f"Backing off for function '{func_name}': "
        f"Attempt {attempt_num} failed due to '{exception.__class__.__name__}: {exception}'. "
        f"Waiting {wait_time:.2f} seconds before next attempt."
    )


@retry(
    wait=wait_random_exponential(min=1, max=60),
    stop=stop_after_attempt(6),
    before_sleep=log_backoff_attempt,
)
async def embedding_with_backoff(**kwargs):
    return await client.embeddings.create(**kwargs)

# Setting up a vector database

[LanceDB documentation](https://lancedb.github.io/lancedb/basic/)

## Creating a LanceDB table

In [6]:
# We'll use the "overview" column as the text to embed
ds[0]

{'budget': 4000000,
 'genres': ['Crime', 'Comedy'],
 'keywords': ['hotel',
  "new year's eve",
  'witch',
  'bet',
  'hotel room',
  'sperm',
  'los angeles',
  'hoodlum',
  'woman director',
  'episode film'],
 'original_language': 'en',
 'overview': "It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'popularity': 22.87623,
 'release_date': datetime.date(1995, 12, 9),
 'revenue': 4300000,
 'runtime': 98.0,
 'spoken_languages': ['English'],
 'title': 'Four Rooms',
 'cast': [{'character': 'Ted the Bellhop', 'name': 'Tim Roth'},
  {'character': 'Man', 'name': 'Antonio Banderas'},
  {'character': 'Angela', 'name': 'Jennifer Beals'},
  {'character': 'Elspeth', 'name': 'Madonna'},
  {'character': 'Margaret', 'name': 'Marisa Tomei'},
  {'character': 'Leo', 'name': 'Bruce Willis'},
  {'character': 'Cheste

In [7]:
overviews = ds["overview"]
print(len(overviews))
overviews[:5]

4800


["It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.',
 "Nemo, an adventurous young clownfish, is unexpectedly taken from his Great Barrier Reef home to a dentist's office aquarium. It's up to his worrisome father Marlin and a friendly but forgetful fish Dory to bring Nemo home -- meeting vegetarian sharks, surfer dude turtles, hypnotic jellyfish, hungry seagulls, and more along the way.",
 "A man with a low IQ has accomplished great things in his life and been present during significant his

In [8]:
ds.features

{'budget': Value(dtype='int64', id=None),
 'genres': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'keywords': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'original_language': Value(dtype='string', id=None),
 'overview': Value(dtype='string', id=None),
 'popularity': Value(dtype='float64', id=None),
 'release_date': Value(dtype='date32', id=None),
 'revenue': Value(dtype='int64', id=None),
 'runtime': Value(dtype='float64', id=None),
 'spoken_languages': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'title': Value(dtype='string', id=None),
 'cast': [{'character': Value(dtype='string', id=None),
   'name': Value(dtype='string', id=None)}],
 'release_year': Value(dtype='int64', id=None)}

## sentence-transformers

In [13]:
overview_embeddings = embedder.encode(overviews)
overview_embeddings[0][:5]

array([ 0.0797189 , -0.22588545, -0.08258644, -0.06734926,  0.07945839],
      dtype=float32)

In [14]:
df = ds.to_pandas()
df["vector"] = overview_embeddings.tolist()
df.iloc[0]

budget               4000000                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [15]:
db = lancedb.connect("./data/lance_db")

In [18]:
tbl = db.create_table("movies", data=df, mode="overwrite")

#### Save results to Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")
!mkdir -p "/content/drive/My Drive/genai4h-wk14"
!cp -r "./data/tmdb-5000-movies" "/content/drive/My Drive/genai4h-wk14/"

### OpenAI

[Slow embeddings?](https://community.openai.com/t/embeddings-api-extremely-slow/1135044)

In [30]:
overviews = ds["overview"]
get_token_count_and_price(overviews, model="text-embedding-3-small")

(311562, 0.00623124)

In [ ]:
_overview_embeddings = await asyncio.gather(
    *[
        embedding_with_backoff(input=overview, model="text-embedding-3-small")
        # client.embeddings.create(input=overview, model="text-embedding-3-small")
        for overview in overviews
    ]
)
overview_embeddings = [
    embedding.data[0].embedding for embedding in _overview_embeddings
]
overview_embeddings[0][:5]

In [ ]:
df = ds.to_pandas()
df["vector"] = overview_embeddings

In [ ]:
db = lancedb.connect("../data/tmdb-5000-movies-openai")
tbl = db.create_table("movies", data=df)

# Retrieval augmented generation (RAG)

* [Vector search](https://lancedb.github.io/lancedb/search/)
* [Hybrid search](https://lancedb.github.io/lancedb/hybrid_search/hybrid_search/)
* [Keyword search](https://lancedb.github.io/lancedb/fts/)  (needs tokenization)

Hybrid and keyword search may not work as well for cross lingual search.

## Load embedding model
<div class="alert alert-block alert-warning">
⚠️ Must use the same model as the one used to create the embeddings
</div>

In [ ]:
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cpu",
)

In [8]:
db = lancedb.connect("./data/lance_db/")
tbl = db.open_table("movies")
tbl.create_fts_index("overview", replace=True)

## Embed the query

In [13]:
q_en = "I want to watch a romantic comedy"
q_zh = "我想看一部浪漫喜劇"

q_en_embedding = embedder.encode(q_en)
q_zh_embedding = embedder.encode(q_zh)
print(len(q_en_embedding), len(q_zh_embedding))
print(q_en_embedding[:5], q_zh_embedding[:5])

384 384
[-0.32662207 -0.63097787  0.14315039 -0.11667649  0.03020003] [-0.23116109 -0.4500395   0.09482061  0.01837618 -0.08190724]


## Querying the database

In [16]:
random_vector = torch.randn(10).numpy()
random_vector

array([ 0.3624173 , -0.39165464,  0.6183053 ,  0.5983386 ,  1.8258102 ,
        0.33074635,  1.4305156 , -0.64978415, -1.469294  ,  1.6759685 ],
      dtype=float32)

<div class="alert alert-block alert-warning">
⚠️ This will fail because the embedding dimension does not match 👇

In [18]:
tbl.search(random_vector).limit(5).to_pandas()

RuntimeError: lance error: Invalid user input: query dim(10) doesn't match the column vector vector dim(384), /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/lance-0.26.0/src/dataset/scanner.rs:756:25

In [22]:
tbl.search(q_en_embedding).select(["overview", "original_title"]).limit(5).to_pandas()

,overview,original_title,_distance
0,"A modern reimagining of the classic romantic comedy, this contemporary version closely follows new love for two couples as they journey from the bar to the bedroom and are eventually put to the test in the real world.",About Last Night,20.804733
1,"Spoof of romantic comedies which focuses on a man (Campbell), his crush (Hannigan), his parents (Coolidge, Willard), and her father (Griffin).",Date Movie,21.273169
2,Romantic comedy. A small town teenager's angst about sexual inexperience drives a comic quest for love and understanding on a birthday to end all birthdays.,16 to Life,22.505936
3,"Dramatic comedy about two unlikely people who find each other while looking for love. Judith Nelson (Holly Hunter) is suddenly single after discovering her husband of fifteen years, a successful doctor (Martin Donovan), has been having an affair with a younger woman. Judith stews, plans, plots and fantasizes, but she can't decide what to do with her life until she goes out to a night club to see singer Liz Bailey (Queen Latifah), who is full of advice on life and love. While out on the town, Judith is suddenly kissed by a total stranger, which opens her eyes to new possibilities ... which is when she notices Pat (Danny De Vito), the elevator operator in her building.",Living Out Loud,24.331312
4,"A romantic comedy centered on Dexter and Emma, who first meet during their graduation in 1988 and proceed to keep in touch regularly. The film follows what they do on July 15 annually, usually doing something together.",One Day,24.688421


In [23]:
tbl.search(q_zh_embedding).select(["overview", "original_title"]).limit(5).to_pandas()

,overview,original_title,_distance
0,"Spoof of romantic comedies which focuses on a man (Campbell), his crush (Hannigan), his parents (Coolidge, Willard), and her father (Griffin).",Date Movie,13.745854
1,"A modern reimagining of the classic romantic comedy, this contemporary version closely follows new love for two couples as they journey from the bar to the bedroom and are eventually put to the test in the real world.",About Last Night,13.997502
2,Romantic comedy. A small town teenager's angst about sexual inexperience drives a comic quest for love and understanding on a birthday to end all birthdays.,16 to Life,15.334779
3,"Dramatic comedy about two unlikely people who find each other while looking for love. Judith Nelson (Holly Hunter) is suddenly single after discovering her husband of fifteen years, a successful doctor (Martin Donovan), has been having an affair with a younger woman. Judith stews, plans, plots and fantasizes, but she can't decide what to do with her life until she goes out to a night club to see singer Liz Bailey (Queen Latifah), who is full of advice on life and love. While out on the town, Judith is suddenly kissed by a total stranger, which opens her eyes to new possibilities ... which is when she notices Pat (Danny De Vito), the elevator operator in her building.",Living Out Loud,16.062124
4,"A sparkling comedic chronicle of a middle-class young man’s romantic misadventures among New York City’s debutante society. Stillman’s deft, literate dialogue and hilariously highbrow observations earned this debut film an Academy Award nomination for Best Original Screenplay. Alongside the wit and sophistication, though, lies a tender tale of adolescent anxiety.",Metropolitan,16.759302
